<a href="https://colab.research.google.com/github/MahdiRahbar/NLP_SLU/blob/main/sentiment/WelshSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
from collections import defaultdict
import math
from typing import List
import os
import re


In [ ]:
fn = os.path.join("..","data","train-v2.tsv")
handle = open(fn, "r")
res = list()
tweets = list()
for i in handle:
    entry = i.strip().split("\t")
    res.append(int(entry[0]))
    tweets.append(entry[1])

In [ ]:
# i tried
def reformat_tweet(tweet):
    #get rid of mentions, links, hashtags and numbers
    tweet2 = re.sub("(@USER|\{URL\}|#\w*|[0-9]+)", "", tweet).strip()
    #lowercase everything
    tweet2 =  tweet2.lower()
    #tokenize
    tweet2 = tweet2.split(" ")
    
    clean_tweet = []
    for word in tweet2:
        #keep only face emojis and alpha
        relevant_piece = re.match("[\w\U00010000-\U0010ffff]+", word)
        if relevant_piece:
            clean_tweet.append(relevant_piece.group(0))
    return clean_tweet

def make_bag_of_words(tweets: List[list]): # list of list here
    bag_of_words = defaultdict(int)  # key-word, val-count
    for tweet in tweets:
        for word in tweet:
            bag_of_words[word] += 1
    return bag_of_words


In [ ]:
class NaiveBayes():
    def __init__(self, xVec: List[list], yVec: list, vocabs: defaultdict(int)):
        self.xVec = xVec #assuming xVec = yVec no missing data
        self.yVec = yVec
        self.classes = set(yVec)

        self.vocabs = vocabs
        self.prior = defaultdict(float)
        self.log_like = defaultdict(lambda: defaultdict(float))

    def _sep_by_class(self):
        big_doc = defaultdict(lambda: defaultdict(int))
        for i in range( len(self.xVec)) :
            c = self.yVec[i]
            for w in self.xVec[i]:
                big_doc[c][w] += 1
        return big_doc #big doc contains word count separated by class {c_1: {word1: count, word2:count,etc}, c_2: {word1: count, etc},...}

    def train(self, smoothing=lambda x: x):
        N_all = len(self.xVec)
        big_doc = self._sep_by_class()

        for c in self.classes:
            N_c = sum(1 for i in self.yVec if i == c)
            self.prior[c] = math.log(N_c / N_all) #calculate prior

            words_in_c = big_doc[c]
            count_all_words_in_c = sum(words_in_c[w] for w in self.vocabs)

            for word in self.vocabs:
                count_word_in_c = words_in_c[word]
                
                # calculate likelihood
                
                self.log_like[word][c] = smoothing(count_word_in_c, count_all_words_in_c)
    def test(self,tweet, impute_for_never_seen=None):
        bestClass = None
        bestLogPrior = None
        for c in self.classes:
            probability = self.prior[c]
            for w in tweet:
                if w in self.vocabs:
                    probability += self.log_like[w][c]
                else:
                    if impute_for_never_seen:
                        probability = impute_for_never_seen
            if bestLogPrior == None:
                bestLogPrior = probability
                bestClass = c
            elif bestLogPrior < probability:
                bestLogPrior = probability
                bestClass = c
        return bestClass


In [ ]:
#test the above method w small example from txtbook
#just want to make sure my math makes sense

#def test_nb():
#    trainingX = ["just plain boring", "entirely predictable and lacks energy",
#                "no surprises and very few laughs", "very powerful", "the most fun film of the summer"]
#    trainingY = [0,0,0,1,1]
#    tweets_reformatted  = reformat_tweets(trainingX)
#    vocabs = make_bag_of_words(tweets_reformatted)
#    nb = NaiveBayes(tweets_reformatted, trainingY, vocabs)
#    print(nb.log_like)
#    nb.train()
#    print(nb.log_like)
#    return( nb.test(["the", "most", "fun"]) )

#divide train-test
tweets_reformatted = list(map(lambda x: reformat_tweet(x), tweets))
train_indexes = range(64000)
trainX = [tweets_reformatted[i] for i in train_indexes]
trainY = [res[i] for i in train_indexes]
vocabs = make_bag_of_words(trainX)
testX = [tweets_reformatted[i] for i in range(64000,80000)]
testY = [res[i] for i in range(64000, 80000)]



In [ ]:
#smoothing add one. im just experimenting some stuff here.
def smoothing_add_one_prob(c_w, count_all_words): return (
    math.log((c_w+1) * (count_all_words+len(vocabs))))
nb = NaiveBayes(trainX, trainY, vocabs)
nb.train(smoothing = smoothing_add_one_prob)
#sum([nb.test(trainX[i]) == trainY[i] for i in range(len(trainX))])/len(trainX)
sum([nb.test(testX[i]) == testY[i] for i in range(len(testX))])/len(testX)


In [ ]:
#smoothing add one. im just experimenting some stuff here.
nb2 = NaiveBayes(trainX, trainY, vocabs)
nb2.train(smoothing=smoothing_add_one_prob)
all_count_ones = sum(nb2.vocabs[i] for i in nb2.vocabs if nb2.vocabs[i] == 1)
impute_for_unknown = all_count_ones/len(vocabs)
#sum([nb.test(trainX[i]) == trainY[i] for i in range(len(trainX))])/len(trainX)
sum([nb.test(testX[i], impute_for_unknown) == testY[i]
    for i in range(len(testX))]) /len(testX)


In [ ]:
#with smoothing

In [ ]:
# this should return either 0 (negative sentiment) or 1 (positive sentiment)
PROVIDED_VOCABS = make_bag_of_words = make_bag_of_words( list(
    map(lambda x: reformat_tweet(x), tweets)) ) #this builds a vocabulary size based on 80k tweets
PRETRAINED_NB = NaiveBayes(trainX, trainY, vocabs)
PRETRAINED_NB.train(smoothing=smoothing_add_one_prob)

def predict_from_scratch(tweet, trained_model= PRETRAINED_NB,vocabs=PROVIDED_VOCABS):
  return trained_model.test(tweet)


In [ ]:
# this should return either 0 (negative sentiment) or 1 (positive sentiment)
def predict_anything_goes(tweet):
  # do something complicated here
  return random.randint(0,1)

In [ ]:
def evaluate():
  total = 0
  correct_from_scratch = 0
  correct_anything_goes = 0
  testfile = open('test.tsv', 'r')
  for line in testfile:
    total += 1
    pieces = line.rstrip("\n").split("\t")
    if predict_from_scratch(pieces[1]) == int(pieces[0]):
      correct_from_scratch += 1
    if predict_anything_goes(pieces[1]) == int(pieces[0]):
      correct_anything_goes += 1
  return (correct_from_scratch/total, correct_anything_goes/total)
  

In [ ]:
evaluate()